###  Planet Analytics API Tutorial

# Summary Statistics: Buildings

## Overview
    
1. [Introduction](#1.-Introduction)
2. [Post a stats job request](#2.-Post-a-stats-job-request)
3. [Get job report results](#3.-Get-job-report-results)
4. [Visualize the time series](#4.-Visualize-the-time-series)
5. [Customize area of interest and time range](#5.-Customize-area-of-interest-and-time-range)

## 1. Introduction

This notebook demonstrates how to request road summary statistics for a subscription using the Anaytics Feeds Stats API and visualize them as time series, enabling further analyses including patterns of life, development trends and anomaly detection.

The workflow involves:
- Posting a stats job request
- Polling the job stats endpoint
- Getting the job report results
- Restructuring the results into a pandas dataframe
- Visualizing the time series

#### Add extra dependencies
This notebook requires hvplot, which may not be available in the main notebook docker image.

In [ ]:
!pip install --quiet hvplot

## 2. Post a stats job request

### a) Check API Connection
_**Note:** If you do not have access to the Analytics Feeds API, you may not be able to run through these examples. Contact [Sales](go.planet.com/getintouch) to learn more._

In [ ]:
import os
import requests

ANALYTICS_BASE_URL = 'https://api.planet.com/analytics/'
# ANALYTICS_BASE_URL = 'https://sif-next.prod.planet-labs.com/'
# change this line if your API key is not set as an env var
API_KEY = os.environ['PL_API_KEY']
# alternatively, you can just set your API key directly as a string variable:
# API_KEY = "YOUR_PLANET_API_KEY_HERE"
# set up a reusable session with required headers
session = requests.Session()
session.headers.update({'content-type':'application/json','Authorization': 'api-key ' + API_KEY})
# make a request to the analytics api
resp = session.get(ANALYTICS_BASE_URL)
if resp.ok:
    print("Yay, you are able to connect to the Planet Analytics API!")
else:
    print("Something is wrong:", resp.content)

### b) Select your subscription
The analytics stats API enables you to create summary stats reports for your analytics subscriptions. You will need the id of a subscription of interest in order to make a stats request.

In [ ]:
import pandas as pd

# Make sure you have a subscription for this buildings feed
FEED_ID = '4cdb1add-2b0a-49fd-9968-54f85bbb6172'
resp = session.get(f"{ANALYTICS_BASE_URL}subscriptions?feedID={FEED_ID}")
if not resp.ok:
    raise Exception('Bad response:', resp.content)

subscriptions = resp.json()['data']
if len(subscriptions) == 0:
    raise Exception(f"You do not have any subscriptions for feed {FEED_ID}")
df = pd.DataFrame.from_records(subscriptions)
df[['id', 'title', 'description', 'startTime', 'endTime']]

In [ ]:
# you can use any of the above subscriptions
subscription = subscriptions[0]

### c) Post a stats report job request to the Analytic Feeds API

In [ ]:
import json
import pprint

request_body = {
    "title": "Stats Demo",
    "subscriptionID": subscription['id'],
    "interval": "month",  # most road and building feeds generate results on a monthly cadence
#     "collection": collection,  # add a geojson feature collection if you want use a custom area of interest
#     "startTime": start_time,  # add custom start time here if desired
#     "endTime": end_time  # add custom end time here if desired
}

stats_post_url = ANALYTICS_BASE_URL + 'stats'

job_post_resp = session.post(
    stats_post_url, 
    data=json.dumps(request_body)
)

pprint.pprint(job_post_resp.json())

### d) Poll the stats endpoint for job completion

In [ ]:
import time

job_link = job_post_resp.json()['links'][0]['href']
status = "pending"
while status != "completed":
    report_status_resp = session.get(
        job_link,
    )
    status = report_status_resp.json()['status']
    print(status)
    time.sleep(2)
    
    
pprint.pprint(report_status_resp.json())

# 3. Get job report results

### a) Get report link from the completed stats job

In [ ]:
report_results_link = report_status_resp.json()['links'][-1]['href']
report_results_link

### b) Get report as json

In [ ]:
results_resp = session.get(
    report_results_link,
)
print(results_resp.status_code)
pprint.pprint(results_resp.json())

### c) Get job report results as a dataframe
The summary stats report can be returned as a csv file. Below, we request the csv and create a pandas dataframe. 

In [ ]:
report_csv_url = report_results_link + '?format=csv'
print(report_csv_url)

In [ ]:
from io import StringIO

csv_resp = session.get(report_csv_url)
data = StringIO(csv_resp.text)
df = pd.read_csv(data)
df.head()

# 4. Visualize the time series

In [ ]:
import holoviews as hv
import hvplot.pandas
from bokeh.models.formatters import DatetimeTickFormatter
hv.extension('bokeh')
formatter = DatetimeTickFormatter(months='%b %Y')

In [ ]:
df.hvplot().options(xformatter=formatter, width=1000)

In [ ]:
df['Building Area Percentage'] = df['Feature Area'] / df['Total Area'] * 100
df['Building Area Percentage'].hvplot().options(xformatter=formatter, width=600)

# 5. Customize area of interest and time range

### a) Pick out an AOI

First take a look at the full subscription AOI:

In [ ]:
from ipyleaflet import Map, GeoJSON
# center an ipyleaflet map around the subscription
geom = subscription['geometry']
if geom['type'] == 'Polygon':
    lon, lat = geom['coordinates'][0][0]
elif geom['type'] == 'MultiPolygon':
    lon, lat = geom['coordinates'][0][0][0]
else:
    print('You may need to re-center the map')
    lon, lat = -122.41, 37.77

m = Map(center=(lat, lon), zoom=8)
# add the subscription geometry to the map
polygon = GeoJSON(data=geom)
m.add_layer(polygon);
m

You can request stats for the entire subscription geometry or for subregions of the subscription geometry. Below we construct a small box inside of the subscription boundary for this demo.

First, convert the subscription boundary to a shapely shape

In [ ]:
import shapely.geometry
aoi_shape = shapely.geometry.shape(subscription['geometry'])
aoi_shape

Now get a bounding box around the subscription geometry:

In [ ]:
print(aoi_shape.bounds)
minx, miny, maxx, maxy = aoi_shape.bounds
bbox = shapely.geometry.box(minx, miny, maxx, maxy)
bbox

Add the bounding box to the map. The bounding box should contain the entire aoi_shape.

In [ ]:
bbox_polygon = GeoJSON(data=shapely.geometry.mapping(bbox), style={'color': 'green', 'opacity': 1, 'fillOpacity': 0.1})
m.add_layer(bbox_polygon);
m

Construct a smaller box that will fit inside of the aoi_shape

In [ ]:
x_diff = maxx - minx
minx2 = minx + x_diff / 5
maxx2 = maxx - x_diff / 5
y_diff = maxy - miny
miny2 = miny + y_diff / 5
maxy2 = maxy - y_diff / 5
smaller_box = shapely.geometry.box(minx2, miny2, maxx2, maxy2)
print(smaller_box.bounds)
smaller_box

Get a custom AOI by taking the interesction of the subscription geometry and the smaller box

In [ ]:
custom_aoi = smaller_box.intersection(aoi_shape)
custom_aoi

Visualize the new custom_aoi on the map

In [ ]:
bbox_polygon = GeoJSON(data=shapely.geometry.mapping(custom_aoi), style={'color': 'red', 'opacity': 1, 'fillOpacity': 0.1})
m.add_layer(bbox_polygon);
m

We used shapely to construct a cusom_aoi. We now need to convert our custom area of interest into geojson again for the api request. 

Alternatively, you can go to geojson.io to draw a custom area of interest on a map and get the geojson representation.

Note: If you don't provide a custom AOI in your stats request, the entire subscription geometry is used.

In [ ]:
import geojson
import pprint
feature = geojson.Feature(geometry=shapely.geometry.mapping(custom_aoi), id="my_custom_box")
collection = geojson.FeatureCollection(features=[feature])
pprint.pprint(collection)

### b) Select a Custom Time Range

In [ ]:
import datetime, dateutil
start_datetime = dateutil.parser.parse(subscription['startTime']) + datetime.timedelta(weeks=4)
# isoformat returns a time with +00:00 and this api requires the Z suffix and no time offset
start_time = start_datetime.isoformat()[:-6] + 'Z'
end_time = subscription['endTime']
print(start_time)
print(end_time)

### c) Request the custom Report

In [ ]:
request_body = {
    "title": "Building Stats Demo - Custom AOI and TOI",
    "subscriptionID": subscription['id'],
    "interval": "month",  # most road and building feeds generate results on a monthly cadence
    "collection": collection,  # this is the custom_aoi as a geojson feature collection,
    "clipToSubscription": True, # use this option if you are ok with the custom AOI being clipped to the subscription boundary
    "startTime": start_time,  # custom start time
    "endTime": end_time  # custom end time
}

job_post_resp = session.post(
    stats_post_url, 
    data=json.dumps(request_body)
)

pprint.pprint(job_post_resp.json())

Poll for job completion

In [ ]:
job_link = job_post_resp.json()['links'][0]['href']
status = "pending"
while status != "completed":
    report_status_resp = session.get(
        job_link,
    )
    status = report_status_resp.json()['status']
    print(status)
    time.sleep(2)

Get the customized stats report as a dataframe

In [ ]:
report_link = [l for l in report_status_resp.json()['links'] if l['rel'] == 'report'][0]['href']
report_csv_url = report_link + '?format=csv'
csv_resp = session.get(report_csv_url)
data = StringIO(csv_resp.text)
df = pd.read_csv(data)
df.head()